In [1]:
import pandas as pd

In [2]:
# Load all datasets
transactions = pd.read_csv("transactions_2019data.csv")
fraud_labels = pd.read_json("fraud_labels.json")
card_info = pd.read_csv("cards_data.csv")
users = pd.read_csv("users_data.csv")

In [3]:
# Display the shape (rows, columns) of each dataset
print("Transactions shape:", transactions.shape)
print("Fraud labels shape:", fraud_labels.shape)
print("Card info shape:", card_info.shape)
print("Users shape:", users.shape)

Transactions shape: (1159966, 12)
Fraud labels shape: (777339, 2)
Card info shape: (6146, 13)
Users shape: (2000, 14)


In [4]:
# Drop sensitive or unnecessary columns from card_info
card_info = card_info.drop(columns=["card_number", "cvv"])

In [5]:
# Remove merchant location-related columns and errors from transactions because they are too high cardinality
transactions = transactions.drop(columns=["merchant_city", "merchant_state", "zip", "errors"])

In [6]:
# Removing redudant and location data to reduce noise.
users = users.drop(columns=["address", "birth_year", "birth_month", "retirement_age", "latitude", "longitude"])

(ChatGPT 5.2, 2025) Prompt: I have two datasets and I want to join them in python, the first dataset has id which corresponds to the second dataset transaction_id how would I do that

In [7]:
# Add 'fraud' flag column to each transaction record.
df_step1 = transactions.merge(
    fraud_labels,
    left_on="id",
    right_on="transaction_id",
    how="left"
)

In [8]:
# Link each transaction to its corresponding card details.
df_step2 = df_step1.merge(
    card_info,
    left_on="card_id",
    right_on="id",
    how="left",
    suffixes=("", "_card")
)

In [9]:
#  Connect transactions to the user who made them.
df_master = df_step2.merge(
    users,
    left_on="client_id",
    right_on="id",
    how="left",
    suffixes=("", "_user")
)


(ChatGPT 5.2, 2025) Prompt:
After merging my datasets, I noticed that several duplicate ID columns appear in df_master, created during the left merges. These include:
transaction_id (duplicate of id)
id_card (duplicate of card_id)
id_user, client_id_card (duplicate of client_id)
Generate the code that only drops these duplicate ID columns without affecting the rest of the dataset.

In [10]:
df_master.dtypes

id                         int64
date                      object
client_id                  int64
card_id                    int64
amount                    object
use_chip                  object
merchant_id                int64
mcc                        int64
transaction_id           float64
fraud                     object
id_card                    int64
client_id_card             int64
card_brand                object
card_type                 object
expires                   object
has_chip                  object
num_cards_issued           int64
credit_limit              object
acct_open_date            object
year_pin_last_changed      int64
card_on_dark_web          object
id_user                    int64
current_age                int64
gender                    object
per_capita_income         object
yearly_income             object
total_debt                object
credit_score               int64
num_credit_cards           int64
dtype: object

In [11]:
# Drop duplicate ID columns
df_master = df_master.drop(columns=["transaction_id", "id_card", "id_user", "client_id_card"])

In [12]:
#renaming id for clarity
df_master = df_master.rename(columns={"id": "transaction_id"})

In [13]:
# Final validation of cleaned dataframe
df_master.info()
df_master.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159966 entries, 0 to 1159965
Data columns (total 25 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   transaction_id         1159966 non-null  int64 
 1   date                   1159966 non-null  object
 2   client_id              1159966 non-null  int64 
 3   card_id                1159966 non-null  int64 
 4   amount                 1159966 non-null  object
 5   use_chip               1159966 non-null  object
 6   merchant_id            1159966 non-null  int64 
 7   mcc                    1159966 non-null  int64 
 8   fraud                  777339 non-null   object
 9   card_brand             1159966 non-null  object
 10  card_type              1159966 non-null  object
 11  expires                1159966 non-null  object
 12  has_chip               1159966 non-null  object
 13  num_cards_issued       1159966 non-null  int64 
 14  credit_limit           1159966 non

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,mcc,fraud,card_brand,...,acct_open_date,year_pin_last_changed,card_on_dark_web,current_age,gender,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,22326462,2019-01-01 00:02:00,496,3186,$119.35,Chip Transaction,30286,4814,No,Visa,...,05/2016,2016,No,47,Male,$17810,$36319,$44737,730,4
1,22326465,2019-01-01 00:05:00,1129,2677,$100.00,Chip Transaction,27092,4829,No,Mastercard,...,04/2011,2011,No,49,Male,$16894,$34449,$36540,686,3
2,22326466,2019-01-01 00:06:00,114,5283,$51.71,Chip Transaction,61195,5541,No,Visa,...,10/2013,2013,No,47,Female,$16892,$34441,$907,725,4
3,22326467,2019-01-01 00:06:00,641,2774,$105.30,Swipe Transaction,75781,5411,No,Visa,...,04/2017,2017,No,62,Male,$18420,$37556,$0,735,6
4,22326468,2019-01-01 00:10:00,114,5283,$82.00,Chip Transaction,61195,5541,No,Visa,...,10/2013,2013,No,47,Female,$16892,$34441,$907,725,4


In [14]:
#reordering the columns for better readability
df_master = df_master[
    [
        "transaction_id", "date", "client_id", "card_id", "amount", "use_chip",
        "merchant_id", "mcc", "fraud",
        "card_brand", "card_type", "has_chip", "num_cards_issued",
        "credit_limit", "acct_open_date", "year_pin_last_changed", "card_on_dark_web",
        "current_age", "gender", "per_capita_income", "yearly_income",
        "total_debt", "credit_score", "num_credit_cards"
    ]
]

df_master.head()


,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,mcc,fraud,card_brand,...,acct_open_date,year_pin_last_changed,card_on_dark_web,current_age,gender,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,22326462,2019-01-01 00:02:00,496,3186,$119.35,Chip Transaction,30286,4814,No,Visa,...,05/2016,2016,No,47,Male,$17810,$36319,$44737,730,4
1,22326465,2019-01-01 00:05:00,1129,2677,$100.00,Chip Transaction,27092,4829,No,Mastercard,...,04/2011,2011,No,49,Male,$16894,$34449,$36540,686,3
2,22326466,2019-01-01 00:06:00,114,5283,$51.71,Chip Transaction,61195,5541,No,Visa,...,10/2013,2013,No,47,Female,$16892,$34441,$907,725,4
3,22326467,2019-01-01 00:06:00,641,2774,$105.30,Swipe Transaction,75781,5411,No,Visa,...,04/2017,2017,No,62,Male,$18420,$37556,$0,735,6
4,22326468,2019-01-01 00:10:00,114,5283,$82.00,Chip Transaction,61195,5541,No,Visa,...,10/2013,2013,No,47,Female,$16892,$34441,$907,725,4


In [15]:
df_master.to_csv("master_dataset.csv")